In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import glob
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.module
import strawb.tools

import plotly.express as px
import pandas
import plotly.io as pio
pio.templates.default = "plotly_white"

import random

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

In [ ]:
# in case execute db.load_entire_db_from_ONC() to load the entire db
db = strawb.SyncDBHandler()  # loads the db
db.load_onc_db_update(output=True, save_db=True)

## Select the files of interest

In [ ]:
timestamp = np.datetime64('2021-09-01T01:00', 'h')  # the time of interst
timerange = np.timedelta64(7*24, "h")  # select files covering the [timestamp, timestamp+timerange]

# Mask files based on 'dateFrom' and 'dateTo' which overlap with the time range
mask = db.date_mask_between(timestamp, timestamp+timerange)

# mask by device and dataProductCode
mask &= db.dataframe.dataProductCode == 'SMRD'
mask &= db.dataframe.deviceCode == 'TUMPMTSPECTROMETER001'

db.dataframe[mask]

### Donwload files, if not all are synced

In [ ]:
### this will download the selected file
if not db.dataframe.synced[mask].all():
    db.update_db_and_load_files(
        db.dataframe[mask],
        output=True,  # print output to console
        download=True,  # download the files
        #save_db=True
    )
    db.save_db()

### Load the file
Handles also multiple files in the timerange. Creats an virtualHDF5 and load it.

In [ ]:
# select the Module file(s) -> dataProductCode == 'MSSCD'
item = db.dataframe[mask & (db.dataframe.dataProductCode == 'SMRD')]

try:  # if the camera file is still open
    module.file_handler.close()
except:
    pass
    
# generate a virtual hdf5 to combine the datasets if there are multiple files selected
if len(item) > 1:
    vhdf5 = strawb.VirtualHDF5('SMRD_event_view.hdf5', item.fullPath.to_list())  
    file_name = vhdf5.file_name
else:
    file_name = item.fullPath[0]
    
module = strawb.sensors.Module(file=file_name)
module.file_handler.file_attributes  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

# Plotting with pandas and plotly

In [ ]:
df_accel = pandas.DataFrame(dict(time = module.file_handler.accel_time.asdatetime()[:],
                                 x    = module.file_handler.accel_x,
                                 y    = module.file_handler.accel_y,
                                 z    = module.file_handler.accel_z,))

In [ ]:
df_magneto = pandas.DataFrame(dict(time = module.file_handler.magneto_time.asdatetime()[:],
                                   x    = module.file_handler.magneto_x,
                                   y    = module.file_handler.magneto_y,
                                   z    = module.file_handler.magneto_z,))

In [ ]:
df_pth = pandas.DataFrame(dict(time        = module.file_handler.pth_time.asdatetime()[:],
                               humidity    = module.file_handler.pth_humidity,
                               temperature = module.file_handler.pth_temperature,
                               pressure    = module.file_handler.pth_pressure,))

In [ ]:
df_temperatures = pandas.DataFrame(dict(time  = module.file_handler.temperatures_time.asdatetime()[:],
                                        temp1 = module.file_handler.temperatures_temp1,
                                        temp2 = module.file_handler.temperatures_temp2,
                                        temp3 = module.file_handler.temperatures_temp3))

In [ ]:
df_pwrmoni = pandas.DataFrame(dict(time           = module.file_handler.pwrmoni_time.asdatetime()[:],
                                   c2_current     = module.file_handler.pwrmoni_c2_current,
                                   c2_voltage     = module.file_handler.pwrmoni_c2_voltage,
                                   laser_current  = module.file_handler.pwrmoni_laser_current,
                                   laser_voltage  = module.file_handler.pwrmoni_laser_voltage,
                                   motor_current  = module.file_handler.pwrmoni_motor_current,
                                   motor_voltage  = module.file_handler.pwrmoni_motor_voltage,
                                   padiwa_current = module.file_handler.pwrmoni_padiwa_current,
                                   padiwa_voltage = module.file_handler.pwrmoni_padiwa_voltage,
                                   switch_current = module.file_handler.pwrmoni_switch_current,
                                   switch_voltage = module.file_handler.pwrmoni_switch_voltage,
                                   trb3sc_current = module.file_handler.pwrmoni_trb3sc_current,
                                   trb3sc_voltage = module.file_handler.pwrmoni_trb3sc_voltage))

In [ ]:
px.line(df_accel, x='time', y=['x', 'y', 'z'])

In [ ]:
px.line(df_magneto, x='time', y=['x', 'y', 'z'])

In [ ]:
px.line(df_pth, x='time', y=['humidity','temperature','pressure'])

In [ ]:
px.line(df_temperatures, x='time', y=['temp1','temp2','temp3'])

In [ ]:
px.line(df_pwrmoni, x='time', y=['c2_current', 'c2_voltage',
                                 'laser_current', 'laser_voltage',
                                 'motor_current', 'motor_voltage',
                                 'padiwa_current', 'padiwa_voltage',
                                 'switch_current', 'switch_voltage',
                                 'trb3sc_current', 'trb3sc_voltage'])

# Plot
## Plot all module sensors

In [ ]:
fig, ax = plt.subplots(nrows=7, squeeze=False, figsize=(7,15), sharex=True)
ax = ax.flatten()

i = 0

# ACCEL
strawb.tools.plot_binned_mean(module.file_handler.accel_time, 
                              module.file_handler.accel_x[:]*9.81, ax=ax[i], label='accel x')
strawb.tools.plot_binned_mean(module.file_handler.accel_time, 
                              module.file_handler.accel_y[:]*9.81, ax=ax[i], label='accel y')
ax[i].set_ylabel('Accel [m/s**2]')
i += 1
strawb.tools.plot_binned_mean(module.file_handler.accel_time, 
                              module.file_handler.accel_z[:]*9.81, ax=ax[i], label='accel z')
ax[i].set_ylabel('Accel [m/s**2]')
i += 1

# Magnet
strawb.tools.plot_binned_mean(module.file_handler.magneto_time, 
                              module.file_handler.magneto_x, ax=ax[i], label='magneto x')
strawb.tools.plot_binned_mean(module.file_handler.magneto_time, 
                              module.file_handler.magneto_y, ax=ax[i], label='magneto y')
strawb.tools.plot_binned_mean(module.file_handler.magneto_time, 
                              module.file_handler.magneto_z, ax=ax[i], label='magneto z')
ax[i].set_ylabel('Magneto [a.u.]')
i += 1

# TEMP.
strawb.tools.plot_binned_mean(module.file_handler.temperatures_time,
                              module.file_handler.temperatures_temp1, ax=ax[i], label='temp 1')
strawb.tools.plot_binned_mean(module.file_handler.temperatures_time,
                              module.file_handler.temperatures_temp2, ax=ax[i], label='temp 2')
ax[i].set_ylabel('Temp [C]')
i += 1
strawb.tools.plot_binned_mean(module.file_handler.temperatures_time, 
                              module.file_handler.temperatures_temp3, ax=ax[i], label='temp 3')
ax[i].set_ylabel('Temp [C]')
# i += 1

# PTH
strawb.tools.plot_binned_mean(module.file_handler.pth_time, 
                              module.file_handler.pth_temperature, ax=ax[i], label='temp pth')
ax[i].set_ylabel('Temp [C]')
i += 1

strawb.tools.plot_binned_mean(module.file_handler.pth_time, 
                              module.file_handler.pth_humidity, ax=ax[i], label='pth humidity')
ax[i].set_ylabel('humidity [per]')
i += 1

strawb.tools.plot_binned_mean(module.file_handler.pth_time, 
                              module.file_handler.pth_pressure, ax=ax[i], label='pth pressure')
ax[i].set_ylabel('pressure [mbar]')
i += 1

for ax_i in ax:
    ax_i.legend(loc=1)
    ax_i.grid()

# # set limits
# limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
fig.autofmt_xdate()
locator = mdates.AutoDateLocator()
#fmt = mdates.AutoDateFormatter(locator, usetex=True)
fmt = mdates.ConciseDateFormatter(locator)
ax[-1].xaxis.set_major_formatter(fmt)
plt.tight_layout()

## Plot current, voltage, power for all subcomponents

In [ ]:
fig, ax = plt.subplots(nrows=3, squeeze=False, figsize=(9,7), sharex=True)
ax = ax.flatten()

i = 0

# ACCEL
for dev_i in module.power.all_devices_list:
    strawb.tools.plot_binned_mean(dev_i.time, 
                                  dev_i.current, 
                                  ax=ax[i], 
                                  label=dev_i.name)

ax[i].set_ylabel('Current [A]')
i += 1

for dev_i in module.power.all_devices_list:
    strawb.tools.plot_binned_mean(dev_i.time, 
                                  dev_i.voltage, 
                                  ax=ax[i], 
                                  label=dev_i.name)

ax[i].set_ylabel('Voltage [V]')
i += 1

for dev_i in module.power.all_devices_list:
    strawb.tools.plot_binned_mean(dev_i.time, 
                                  dev_i.watt, 
                                  ax=ax[i], 
                                  label=dev_i.name)

ax[i].set_ylabel('Power [W]')
i += 1

for ax_i in ax:
    ax_i.grid()
    ax_i.set_ylim(0)
    
ax[0].legend(loc='lower center', bbox_to_anchor=(.5, 1.0), ncol=len(module.power.all_devices_list))

# # set limits
# limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
fig.autofmt_xdate()
locator = mdates.AutoDateLocator()
#fmt = mdates.AutoDateFormatter(locator, usetex=True)
fmt = mdates.ConciseDateFormatter(locator)
ax[-1].xaxis.set_major_formatter(fmt)
ax[-1].autoscale(enable=True, axis='x', tight=True)
plt.tight_layout()